In [42]:
import os
import shutil
import hashlib

## Deduplicate Assignemnts

In [47]:
DATA_PATH = "data/raw"

hashes = {}

for root, _, files in os.walk(DATA_PATH):
    for file in files:
        with open(os.path.join(root, file), "rb") as f:
            digest = hashlib.file_digest(f, "sha256")

        dhash = digest.hexdigest()

        if dhash in hashes.keys():
            hashes[dhash].append(os.path.join(root, file))
        else:
            hashes[dhash] = [os.path.join(root, file)]

## Extract files for jar

In [52]:
PROCESSED_PATH = "data/proc"

temp = False

for root, _, files in os.walk(DATA_PATH):
    for file in files:
        if file.endswith(".jar"):
            path = os.path.join(root, file).replace(" ", '\ ')
            print(root.replace(DATA_PATH, PROCESSED_PATH))
            # os.system('unzip ' + path + " -d ./" + )

            temp = True
            break

    if temp:
        break

data/proc/4CCS1PPA 18~19 000001 PROGRAMMING PRA-Assignment 3 Code Submission-2410129/Stefanos Karathanassis_2742361_assignsubmission_file_


## Remove unnecessary files

In [53]:
for root, subdirs, files in os.walk(DATA_PATH):

    for subdir in subdirs:
        if subdir.startswith("."):
            shutil.rmtree(os.path.join(root, subdir))

    for file in files:
        if file.endswith(".class") or file.endswith(".ctxt") or file.endswith("~"):
            os.remove(os.path.abspath(os.path.join("./", root, file)))